# Keras 神经网络剪枝

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/g3doc/guide/pruning/pruning_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## 环境配置

In [0]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly
! pip install -U tf-nightly-gpu

! pip install tensorflow-model-optimization

In [1]:
%load_ext tensorboard
import tensorboard

The tensorboard module is not an IPython extension.


In [2]:
import tensorflow as tf
tf.enable_eager_execution()

import tempfile
import zipfile
import os

## 准备训练数据

In [ ]:
batch_size = 128
num_classes = 10
epochs = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

if tf.keras.backend.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## 使用未剪枝模型训练MNIST 模型

### Build the MNIST model

In [0]:
l = tf.keras.layers

model = tf.keras.Sequential([
    l.Conv2D(
        32, 5, padding='same', activation='relu', input_shape=input_shape),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.BatchNormalization(),
    l.Conv2D(64, 5, padding='same', activation='relu'),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.Flatten(),
    l.Dense(1024, activation='relu'),
    l.Dropout(0.4),
    l.Dense(num_classes, activation='softmax')
])

model.summary()

### Train the model to reach an accuracy >99%


Load [TensorBoard](https://www.tensorflow.org/tensorboard) to monitor the training process

In [0]:
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

In [0]:
%tensorboard --logdir={logdir}

In [0]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Save the original model for size comparison later

In [0]:
# Backend agnostic way to save/restore models
_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

## 训练剪枝模型

我们提供了一个prune_low_magnitude（）API来训练已删除连接的模型。

例如，典型配置将通过从步骤2,000开始每100个步骤（也称为epochs）修剪连接来瞄准75％的稀疏度。 有关可能配置的更多详细信息，请参阅github文档。

### 逐层构建剪枝模型
在此示例中，我们将展示如何在层级使用API，并构建修剪的MNIST模型。

在这种情况下，`prune_low_magnitude（`）
接收我们想要修剪其重量的Keras层作为参数。

此功能需要修剪参数，在训练期间配置修剪算法。 有关详细文档，请参阅我们的github页面。 这里使用的参数表示：


1.   **Sparsity.** PolynomialDecay is used across the whole training process. We start at the sparsity level 50% and gradually train the model to reach 90% sparsity. X% sparsity means that X% of the weight tensor is going to be pruned away.
2.   **Schedule**. Connections are pruned starting from step 2000 to the end of training, and runs every 100 steps. The reasoning behind this is that we want to train the model without pruning for a few epochs to reach a certain accuracy, to aid convergence. Furthermore, we give the model some time to recover after each pruning step, so pruning does not happen on every step. We set the pruning frequency to 100.



In [0]:
from tensorflow_model_optimization.sparsity import keras as sparsity

为了演示如何保存和恢复修剪的keras模型，在下面的示例中，我们首先训练模型10个epochs，将其保存到磁盘，最后恢复并继续训练2个epochs。 逐渐稀疏，四个重要参数是begin_sparsity，final_sparsity，begin_step和end_step。 前三个是直截了当的。 让我们根据训练样本数量，批量大小和训练的总时期来计算结束步骤。

In [0]:
import numpy as np

epochs = 12
num_train_samples = x_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

In [0]:
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=2000,
                                                   end_step=end_step,
                                                   frequency=100)
}

pruned_model = tf.keras.Sequential([
    sparsity.prune_low_magnitude(
        l.Conv2D(32, 5, padding='same', activation='relu'),
        input_shape=input_shape,
        **pruning_params),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.BatchNormalization(),
    sparsity.prune_low_magnitude(
        l.Conv2D(64, 5, padding='same', activation='relu'), **pruning_params),
    l.MaxPooling2D((2, 2), (2, 2), padding='same'),
    l.Flatten(),
    sparsity.prune_low_magnitude(l.Dense(1024, activation='relu'),
                                 **pruning_params),
    l.Dropout(0.4),
    sparsity.prune_low_magnitude(l.Dense(num_classes, activation='softmax'),
                                 **pruning_params)
])

pruned_model.summary()

Load Tensorboard

In [0]:
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

In [0]:
%tensorboard --logdir={logdir}

### 训练模型

Start pruning from step 2000 when accuracy >98%

In [0]:
pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

pruned_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))

score = pruned_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Save and restore the pruned model

Continue training for two epochs:

In [0]:
_, checkpoint_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', checkpoint_file)
# saved_model() sets include_optimizer to True by default. Spelling it out here
# to highlight.
tf.keras.models.save_model(pruned_model, checkpoint_file, include_optimizer=True)

with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(checkpoint_file)

restored_model.fit(x_train, y_train,
                   batch_size=batch_size,
                   epochs=2,
                   verbose=1,
                   callbacks=callbacks,
                   validation_data=(x_test, y_test))

score = restored_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

在上面的示例中，需要注意的一些事项是：


*   保存模型时，必须将include_optimizer设置为True。 我们需要在训练期间保留优化器的状态，以便修剪工作正常。
*   加载修剪后的模型时，需要使用prune_scope()进行反序列化。



### 在发布模型之前，从修剪过的模型中去除pruning wrappers 
在导出服务模型之前，您需要调用`strip_pruning` API来从模型中剥离修剪包装器，因为它只需要进行培训。

In [0]:
final_model = sparsity.strip_pruning(pruned_model)
final_model.summary()

In [0]:
_, pruned_keras_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', pruned_keras_file)

# No need to save the optimizer with the graph for serving.
tf.keras.models.save_model(final_model, pruned_keras_file, include_optimizer=False)

### 比较压缩后未修剪模型与修剪模型的大小

In [0]:
_, zip1 = tempfile.mkstemp('.zip') 
with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(keras_file)
print("Size of the unpruned model before compression: %.2f Mb" % 
      (os.path.getsize(keras_file) / float(2**20)))
print("Size of the unpruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip1) / float(2**20)))

_, zip2 = tempfile.mkstemp('.zip') 
with zipfile.ZipFile(zip2, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" % 
      (os.path.getsize(pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip2) / float(2**20)))



### 修剪整个模型

`prune_low_magnitude`函数也可以应用于整个Keras模型。

在这种情况下，该算法将应用于所有可用于权重修剪的层（API知道）。 API知道的不能用于权重修剪的图层将被忽略，API的未知图层将导致错误。

*如果您的模型具有API不知道如何修剪其权重的层，但是完全可以保留“未修剪”，那么只需在每层中应用API。*

关于修剪配置，相同的设置适用于模型中的所有可修剪层。

另外值得注意的是，修剪不会保留与原始模型关联的优化程序。 因此，有必要使用新的优化器重新编译已修剪的模型。 

在我们继续这个例子之前，让我们讨论一下您可能已经拥有序列化预训练Keras模型的常见用例，您希望对其进行权重修剪。 我们将采用之前训练过的原始MNIST模型来展示其工作原理。 在这种情况下，您首先将模型加载到内存中，如下所示：

In [0]:
# Load the serialized model
loaded_model = tf.keras.models.load_model(keras_file)

然后你可以修剪加载的模型并编译修剪后的模型进行训练。 在这种情况下，训练将从步骤0重新开始。鉴于我们加载的模型已达到令人满意的准确度，我们可以立即开始修剪。 结果，我们在这里将begin_step设置为0，并且仅训练另外四个epochs.

In [0]:
epochs = 4
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

Load tensorboard

In [0]:
logdir = tempfile.mkdtemp()
print('Writing training logs to ' + logdir)

In [0]:
%tensorboard --logdir={logdir}

### 将模型训练另外四个epochs

In [0]:
# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

new_pruned_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))

score = new_pruned_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### 导出已修剪的模型

In [0]:
final_model = sparsity.strip_pruning(pruned_model)
final_model.summary()

In [0]:
_, new_pruned_keras_file = tempfile.mkstemp('.h5')
print('Saving pruned model to: ', new_pruned_keras_file)
tf.keras.models.save_model(final_model, new_pruned_keras_file, 
                        include_optimizer=False)

压缩后的模型尺寸与逐层修剪的模型尺寸相同

In [0]:
_, zip3 = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip3, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(new_pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" 
      % (os.path.getsize(new_pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" 
      % (os.path.getsize(zip3) / float(2**20)))

## 转换为TensorFlow Lite

最后，您可以将已修剪的模型转换为可在目标后端运行的格式。 Tensorflow Lite是可用于部署到移动设备的示例格式。 要转换为Tensorflow Lite图形，您需要使用TFLiteConverter，如下所示：

### 使用TFLiteConverter转换模型

In [0]:
tflite_model_file = '/tmp/sparse_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

压缩后TensorFlow Lite模型的大小

In [0]:
_, zip_tflite = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

### 评估TensorFlow Lite模型的准确性

In [0]:
import numpy as np

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 28, 28, 1))
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))

  return float(num_correct) / float(total_seen)

print(eval_model(interpreter, x_test, y_test))

### 量化TensorFlow Lite模型

您可以将修剪与其他优化技术（如训练后量化）结合起来。 作为回顾，训练后量化将权重转换为8位精度，作为从keras模型到TFLite平坦缓冲区的模型转换的一部分，从而使模型大小减少4倍。

在下面的示例中，我们采用修剪的keras模型，使用训练后量化转换它，检查尺寸减小并验证其准确性。

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

tflite_quant_model_file = '/tmp/sparse_mnist_quant.tflite'
with open(tflite_quant_model_file, 'wb') as f:
  f.write(tflite_quant_model)

In [0]:
_, zip_tflite = tempfile.mkstemp('.zip')
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_quant_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_quant_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

量化模型的大小约为原始模型的1/4。

In [0]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(eval_model(interpreter, x_test, y_test))

## Conclusion

In this tutorial, we showed you how to create *sparse models* with the TensorFlow model optimization toolkit weight pruning API. Right now, this allows you to create models that take significant less space on disk. The resulting model can also be more efficiently implemented to avoid computation; in the future TensorFlow Lite will provide such capabilities.

More specifically, we walked you through an end-to-end example of training a simple MNIST model that used the weight pruning API. We showed you how to convert it to the Tensorflow Lite format for mobile deployment, and demonstrated how with simple file compression the model size was reduced 5x.

We encourage you to try this new capability on your Keras models, which can be particularly important for deployment in resource-constraint environments. 

